In [46]:
import pandas as pd
import datetime

In [73]:
# importing news article csv files and cleaning them
fb_articles_df = pd.read_csv('fb_articles_20180822_20180829_df.csv', index_col = 0)
fb_articles_df['publishedAt'] = pd.to_datetime(fb_articles_df['publishedAt']) # set dtype to datetime
fb_articles_df['publishedAt'] = fb_articles_df['publishedAt'].dt.round('30min') 
fb_articles_df = fb_articles_df.set_index(['publishedAt'])

# importing tweet csv files and cleaning them
fb_tweets_df = pd.read_csv('tweets_0822_0829_final.csv')
fb_tweets_df['time_stamp'] = pd.to_datetime(fb_tweets_df['time_stamp'])
fb_tweets_df['time_stamp'] = fb_tweets_df['time_stamp'].dt.round('30min')
fb_tweets_df = fb_tweets_df.set_index(['time_stamp'])

# reading stock.csv and cleaning the df
running_stocks_thirty = pd.read_csv('stock.csv', index_col = 0)
running_stocks_thirty.index = pd.to_datetime(running_stocks_thirty.index)

In [ ]:
df.groupby('Company Name')['Amount'].agg(['sum','count'])

In [110]: (df.groupby('Company Name')
   .....:    .agg({'Organisation Name':'count', 'Amount': 'sum'})
   .....:    .reset_index()
   .....:    .rename(columns={'Organisation Name':'Organisation Count'})
   .....: )

In [96]:
fb_tweets_master_df.head()

,vader_sentiment,sentiment
time_stamp,,
2018-08-29 20:00:00,0.0000,neutral
2018-08-29 20:00:00,0.4019,pos
2018-08-29 20:00:00,0.0000,neutral
2018-08-29 20:00:00,-0.6468,neg
2018-08-29 20:00:00,0.7096,pos


In [116]:
fb_tweets_master_df = fb_tweets_df.drop(['t_id', 'text'], axis = 1)
sent_avg = fb_tweets_master_df.groupby(fb_tweets_master_df.index).agg({'vader_sentiment': 'mean'}).rename(columns = {'vader_sentiment': 'sentiment_avg_30'})
sent_count = fb_tweets_master_df.groupby(fb_tweets_master_df.index).agg({'sentiment': 'value_counts'}).unstack().rename(columns = {'sentiment': 'sentiment_30'})
fb_tweets_master_df = pd.concat([sent_avg, sent_count], axis = 1)

In [123]:
open_hours_tweets = fb_tweets_master_df.between_time('9:30', "15:30")
after_hour_tweets = fb_tweets_master_df.between_time('15:30', "9:30")

In [127]:
open_hours_tweets

,sentiment_avg_30,"(sentiment_30, neg)","(sentiment_30, neutral)","(sentiment_30, pos)"
time_stamp,,,,
2018-08-22 14:00:00,-0.113600,1.0,1.0,1.0
2018-08-22 14:30:00,0.022714,34.0,51.0,41.0
2018-08-22 15:00:00,0.091563,22.0,58.0,48.0
2018-08-22 15:30:00,0.085710,31.0,43.0,44.0
2018-08-23 12:30:00,-0.000305,15.0,11.0,13.0
2018-08-23 13:00:00,0.043919,30.0,45.0,43.0
2018-08-23 13:30:00,0.013543,38.0,49.0,36.0
2018-08-23 14:00:00,-0.037498,54.0,51.0,43.0
2018-08-23 14:30:00,0.022061,45.0,69.0,50.0


In [ ]:
open_hours_tweets

In [134]:
open_hours_tweets_hour = open_hours_tweets.shift().rolling(window=2, center = True, min_periods=2).mean().rename(columns = {'sentiment_avg_30': 'sentiment_avg_60', '(sentiment_30, neg)': '(sentiment_60, neg)'})

In [132]:
open_hours_tweets.shift()

,sentiment_avg_30,"(sentiment_30, neg)","(sentiment_30, neutral)","(sentiment_30, pos)"
time_stamp,,,,
2018-08-22 14:00:00,NaN,NaN,NaN,NaN
2018-08-22 14:30:00,-0.113600,1.0,1.0,1.0
2018-08-22 15:00:00,0.022714,34.0,51.0,41.0
2018-08-22 15:30:00,0.091563,22.0,58.0,48.0
2018-08-23 12:30:00,0.085710,31.0,43.0,44.0
2018-08-23 13:00:00,-0.000305,15.0,11.0,13.0
2018-08-23 13:30:00,0.043919,30.0,45.0,43.0
2018-08-23 14:00:00,0.013543,38.0,49.0,36.0
2018-08-23 14:30:00,-0.037498,54.0,51.0,43.0


In [117]:
fb_tweets_master_df

,sentiment_avg_30,"(sentiment_30, neg)","(sentiment_30, neutral)","(sentiment_30, pos)"
time_stamp,,,,
2018-08-22 14:00:00,-0.113600,1.0,1.0,1.0
2018-08-22 14:30:00,0.022714,34.0,51.0,41.0
2018-08-22 15:00:00,0.091563,22.0,58.0,48.0
2018-08-22 15:30:00,0.085710,31.0,43.0,44.0
2018-08-22 16:00:00,0.054309,34.0,63.0,53.0
2018-08-22 16:30:00,0.040894,34.0,68.0,39.0
2018-08-22 17:00:00,0.046090,25.0,65.0,31.0
2018-08-22 17:30:00,0.129835,25.0,55.0,65.0
2018-08-22 18:00:00,0.062002,29.0,83.0,50.0


In [83]:
running_stocks_thirty

,1. open,2. high,3. low,4. close,5. volume
2018-08-29 15:30:00,176.0000,176.0300,175.5601,175.9600,1626278
2018-08-29 15:00:00,175.2800,176.0600,175.2600,176.0000,1585608
2018-08-29 14:30:00,174.9800,175.3000,174.9200,175.2600,1020249
2018-08-29 14:00:00,174.9839,175.1050,174.8600,174.9800,876414
2018-08-29 13:30:00,175.3434,175.3500,174.7500,174.9700,1349878
2018-08-29 13:00:00,175.6100,175.6498,175.1850,175.3400,662712
2018-08-29 12:30:00,175.5500,175.7599,175.3600,175.6300,865557
2018-08-29 12:00:00,175.8800,175.9200,175.4400,175.5251,709158
2018-08-29 11:30:00,176.0000,176.1800,175.7600,175.8700,767317
2018-08-29 11:00:00,175.5400,176.1107,175.5400,176.0315,1073041


In [81]:
set(running_stocks_thirty.index.values)

{numpy.datetime64('2018-08-20T11:30:00.000000000'),
 numpy.datetime64('2018-08-20T12:00:00.000000000'),
 numpy.datetime64('2018-08-20T12:30:00.000000000'),
 numpy.datetime64('2018-08-20T13:00:00.000000000'),
 numpy.datetime64('2018-08-20T13:30:00.000000000'),
 numpy.datetime64('2018-08-20T14:00:00.000000000'),
 numpy.datetime64('2018-08-20T14:30:00.000000000'),
 numpy.datetime64('2018-08-20T15:00:00.000000000'),
 numpy.datetime64('2018-08-20T15:30:00.000000000'),
 numpy.datetime64('2018-08-21T09:30:00.000000000'),
 numpy.datetime64('2018-08-21T10:00:00.000000000'),
 numpy.datetime64('2018-08-21T10:30:00.000000000'),
 numpy.datetime64('2018-08-21T11:00:00.000000000'),
 numpy.datetime64('2018-08-21T11:30:00.000000000'),
 numpy.datetime64('2018-08-21T12:00:00.000000000'),
 numpy.datetime64('2018-08-21T12:30:00.000000000'),
 numpy.datetime64('2018-08-21T13:00:00.000000000'),
 numpy.datetime64('2018-08-21T13:30:00.000000000'),
 numpy.datetime64('2018-08-21T14:00:00.000000000'),
 numpy.datet